In [2]:
using TensorDecomposition, SymPy, LinearAlgebra, Combinatorics, InvertedIndices

In [3]:
function multMon2(x, i)
    y = [j for j in x]
    
    y[1] -= 1
    y[i] += 1
    return Tuple(j for j in y)
end

function alpha_iterator(::Val{N}, s, t=()) where {N}
    N <= 1 && return ((s, t...),) # Iterator over a single Tuple
    Iterators.flatten(alpha_iterator(Val(N-1), s-i, (i, t...)) for i in 0:s)
end

function monomialOrder(a, b)
    if a[1] > b[1]
        return true
    elseif a[1] < b[1]
        return false
    else 
        if maximum(a[2:end]) < maximum(b[2:end])
            return true
        elseif maximum(a[2:end]) > maximum(b[2:end])
            return false
        else
            return !isless(a[2:end], b[2:end])
        end 
    end
end;

function cofactor(A::AbstractMatrix)
    ax = axes(A)
    out = similar(A, eltype(A), ax)
    for col in ax[1]
        for row in ax[2]
            out[col, row] = (-1)^(col + row) * det(A[InvertedIndices.Not(col), InvertedIndices.Not(row)])
        end
    end
    return out
end;

function convertIndices(x)
    d = sum(x)
    y = zeros(Int, d)
    c = 1
    for i=1:length(x)
        pow = x[i]
        y[c:c+pow-1] .= i
        c += pow
    end 
    return y .- 1
end;

In [4]:
n = 3
d = 6;

In [5]:
r = 4;

In [6]:
Fdim = binomial(n-1+d, d);

In [7]:
a = symbols("a[1:16]");

In [8]:
D = Dict(i => j for (i, j) in zip(with_replacement_combinations(1:n, d), a));

In [9]:
T = zeros(Sym, (n, n, n, n))
for ind in Base.product(1:n, 1:n, 1:n, 1:n)
    T[ind...] = D[sort([x for x in ind])]
end

In [10]:
Fliftdim = binomial(n-1+2*d, n-1)

F = zeros(Sym, Fdim)
alphas = sort(collect(alpha_iterator(Val(n), d)), lt=monomialOrder)
for (i, ind) in enumerate(convertIndices.(alphas))
    F[i] = T[(ind .+ 1)...]
end

Flift = Array{Sym}(undef, Fliftdim)
h =  symbols("h[1:31]");

In [11]:
Thank = Array{Sym}(undef, Fdim, Fdim)

for i=1:length(F)
    Flift[i] = F[i]
end


for i=1:Fliftdim-Fdim
    Flift[Fdim+i] = h[i]
end

alphas2 = sort(collect(alpha_iterator(Val(n), 2*d)), lt=monomialOrder)
D = Dict()
for (i, ind) in enumerate(alphas2)
    D[ind] = i
end

for (i, row_ind) in enumerate(alphas)
    for (j, col_ind) in enumerate(alphas)
        row_mon = [x for x in row_ind]
        col_mon = [x for x in col_ind]
        ind = Tuple(x for x in row_mon+col_mon)
        Thank[i, j] = Flift[D[ind]]
    end
end

In [12]:
H0 = Thank[1:r, 1:r];
# H0_inv = inv(H0);
H0_adj = cofactor(H0);
H0_det = det(H0);

In [13]:
alphas = sort(collect(alpha_iterator(Val(n), d)), lt=monomialOrder)
D = Dict()
first_r = []
for (i, ind) in enumerate(alphas)
    if i <= r
        push!(first_r, ind)
    end
    D[ind] = i
end

Hs = []
for i=2:n
    hankInds = []
    multMap = map(x -> multMon2(x, i), first_r)
    for ind in multMap
        push!(hankInds, D[ind])
    end
    push!(Hs, Thank[1:r, hankInds])
end

In [17]:
# Ms = []
# for H in Hs
#     push!(Ms, H*H0_inv)
# end;

In [15]:
Ms = []
for H in Hs
    push!(Ms, H*H0_adj)
end

In [16]:
vars = [h[1], h[2]];

In [18]:
eqMat1 = Hs[1]*H0_adj*Hs[2];
eqMat2 = Hs[2]*H0_adj*Hs[1];
eqMat = eqMat1-eqMat2;

In [19]:
expand((Hs[1]*H0_adj)[1, 2])

                                      2             2                         
a[13]⋅a[1]⋅a[4]⋅a[6] - a[13]⋅a[1]⋅a[5]  - a[13]⋅a[2] ⋅a[6] + 2⋅a[13]⋅a[2]⋅a[3]

                  2                      2                                    
⋅a[5] - a[13]⋅a[3] ⋅a[4] - a[1]⋅a[4]⋅a[9]  + 2⋅a[1]⋅a[5]⋅a[8]⋅a[9] - a[1]⋅a[6]

     2       2     2                                      2                   
⋅a[8]  + a[2] ⋅a[9]  - 2⋅a[2]⋅a[3]⋅a[8]⋅a[9] - 2⋅a[2]⋅a[5] ⋅a[9] + 2⋅a[2]⋅a[5]

                 2     2                                      2               
⋅a[6]⋅a[8] + a[3] ⋅a[8]  + 2⋅a[3]⋅a[4]⋅a[5]⋅a[9] - 2⋅a[3]⋅a[5] ⋅a[8] - a[4]⋅a[

  2            4
5] ⋅a[6] + a[5] 

In [20]:
expand((Hs[1]*H0_adj)[3, 4])

                                      2             2                         
a[13]⋅a[1]⋅a[4]⋅a[6] - a[13]⋅a[1]⋅a[5]  - a[13]⋅a[2] ⋅a[6] + 2⋅a[13]⋅a[2]⋅a[3]

                  2                      2                                    
⋅a[5] - a[13]⋅a[3] ⋅a[4] - a[1]⋅a[4]⋅a[9]  + 2⋅a[1]⋅a[5]⋅a[8]⋅a[9] - a[1]⋅a[6]

     2       2     2                                      2                   
⋅a[8]  + a[2] ⋅a[9]  - 2⋅a[2]⋅a[3]⋅a[8]⋅a[9] - 2⋅a[2]⋅a[5] ⋅a[9] + 2⋅a[2]⋅a[5]

                 2     2                                      2               
⋅a[6]⋅a[8] + a[3] ⋅a[8]  + 2⋅a[3]⋅a[4]⋅a[5]⋅a[9] - 2⋅a[3]⋅a[5] ⋅a[8] - a[4]⋅a[

  2            4
5] ⋅a[6] + a[5] 

In [21]:
expand((Hs[2]*H0_adj)[1, 3])

                                      2             2                         
a[13]⋅a[1]⋅a[4]⋅a[6] - a[13]⋅a[1]⋅a[5]  - a[13]⋅a[2] ⋅a[6] + 2⋅a[13]⋅a[2]⋅a[3]

                  2                      2                                    
⋅a[5] - a[13]⋅a[3] ⋅a[4] - a[1]⋅a[4]⋅a[9]  + 2⋅a[1]⋅a[5]⋅a[8]⋅a[9] - a[1]⋅a[6]

     2       2     2                                      2                   
⋅a[8]  + a[2] ⋅a[9]  - 2⋅a[2]⋅a[3]⋅a[8]⋅a[9] - 2⋅a[2]⋅a[5] ⋅a[9] + 2⋅a[2]⋅a[5]

                 2     2                                      2               
⋅a[6]⋅a[8] + a[3] ⋅a[8]  + 2⋅a[3]⋅a[4]⋅a[5]⋅a[9] - 2⋅a[3]⋅a[5] ⋅a[8] - a[4]⋅a[

  2            4
5] ⋅a[6] + a[5] 

In [25]:
expand.(eqMat1[2, 3])

                                                                           2  
-a[10]⋅a[12]⋅a[1]⋅a[4]⋅a[9] + a[10]⋅a[12]⋅a[1]⋅a[5]⋅a[8] + a[10]⋅a[12]⋅a[2] ⋅a

                                                        2                     
[9] - a[10]⋅a[12]⋅a[2]⋅a[3]⋅a[8] - a[10]⋅a[12]⋅a[2]⋅a[5]  + a[10]⋅a[12]⋅a[3]⋅a

                                                                              
[4]⋅a[5] + a[10]⋅a[13]⋅a[1]⋅a[4]⋅a[8] - a[10]⋅a[13]⋅a[1]⋅a[5]⋅a[7] - a[10]⋅a[1

       2                                                                      
3]⋅a[2] ⋅a[8] + a[10]⋅a[13]⋅a[2]⋅a[3]⋅a[7] + a[10]⋅a[13]⋅a[2]⋅a[4]⋅a[5] - a[10

                 2                                              3             
]⋅a[13]⋅a[3]⋅a[4]  + a[10]⋅a[1]⋅a[7]⋅a[8]⋅a[9] - a[10]⋅a[1]⋅a[8]  - a[10]⋅a[2]

                                                                    2         
⋅a[4]⋅a[8]⋅a[9] - a[10]⋅a[2]⋅a[5]⋅a[7]⋅a[9] + 2⋅a[10]⋅a[2]⋅a[5]⋅a[8]  + a[10]⋅

              2                               